# Skill assessment (CRPS calculations)

Since the seasonal forecasts are comprised of ensemble members, generally applied simple evaluation methods such as RMSE (Root Mean Squared Error) and R2 have limitations. 
Thus, in this code, CRPS or CRPSS are adopted to measure the skill of ensembled datasets.
This code enables you to calculate those indices automatically.

## 1. Import libraries
Now, we need to import the necessary libraries and tools (🚨 in order to run the code like in the box below, place the mouse pointer in the cell, then click on “run cell” button above or press shift + enter).

In [47]:
import os, re
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import hydrostats.ens_metrics as em

## 2. Settings

In [48]:
# Define the forecast provider
forecast_center = 'ECMWF'

# Assign working directory and time series data
path = os.getcwd()

# Input simulation information
catchment_name = 'A'
start_year = 2011
start_month = 1
start_day = 1
start_date = str(start_month).zfill(2) + '/' + str(start_day).zfill(2) + '/' + str(start_year)
end_year = 2020
end_month = 12
end_day = 31
end_date = str(end_month).zfill(2) + '/' + str(end_day).zfill(2) + '/' + str(end_year)

##  3. Rearrange Datasets

Basically, CRPS/CRPSS are calculated according to lead time. Therefore, we need to collect them into a single file for each lead time. This code enable to collect every data having same lead time. 

### 3.1 ESP ensemble

In [55]:
for leadtime in range(1, 8):  # Loop over lead times from 1 to 7 months
    for year in range(start_year, end_year + 1):  # Loop over years
        for month in range(start_month, end_month + 1):  # Loop over months within the specified range

            # Read ESP simulation results for the current year and month
            df = pd.read_csv(path + '/analysis/3.ESP/3_run/[out]' + catchment_name + '_' + str(year) + '_' 
                             + str(month).zfill(2) + '.csv')

            # Calculate cumulative sum of mean values grouped by lead time
            df2 = df.groupby(by=['leadtime']).mean().cumsum()

            # Create a 'date' column containing the current year and month
            df2['date'] = str(year) + '_' + str(month)

            # Rearrange column order with 'date' as the first column
            col1 = df2.columns[-1:].to_list()
            col2 = df2.columns[:-1].to_list()
            new_col = col1 + col2
            df3 = df2[new_col]

            # Extract data for the current lead time and transpose it
            temp = pd.DataFrame(df3.loc[leadtime]).T

            # Stack data for the first iteration of the outer loop (initialize temp1)
            if year == int(start_year) and month == int(start_month):
                temp1 = temp
            else:
                pass

            # Concatenate current iteration's data with existing temp1 DataFrame
            temp1 = pd.concat([temp1, temp], axis=0, ignore_index=True)

    # Remove the first row (header) and set 'date' as the index
    temp1 = temp1.iloc[1:]
    temp1.set_index('date', inplace=True)

    # Save the aggregated data to a CSV file under the 'skill' directory for each lead time
    temp1.to_csv(path + '/analysis/3.ESP/3_run/skill/' + catchment_name + '_' + str(leadtime) + '_esp.csv')


### 3.2 SFFs ensemble

In [50]:
folder = {1: 'original', 2: 'biascorrected'}

for bc_type in range(1, 3):  # Loop through bias correction types (1 for original, 2 for bias corrected)
    for leadtime in range(1, 8):  # Loop through lead times from 1 to 7 months
        for year in range(start_year, end_year + 1):  # Loop through years within the specified range
            for month in range(start_month, end_month + 1):  # Loop through months within the specified range
                
                # Read seasonal hydrological forecasts results from CSV files
                df = pd.read_csv(path + '/analysis/4.SFFs/3_run/' + folder[bc_type] + '/[out]' + catchment_name + '_' 
                                 + str(year) + '_' + str(month).zfill(2) + '.csv')

                # Calculate cumulative sum of mean values grouped by lead time
                df2 = df.groupby(by=['leadtime']).mean().cumsum()

                # Create a 'date' column containing the current year and month
                df2['date'] = str(year) + '_' + str(month)

                # Rearrange column order with 'date' as the first column
                col1 = df2.columns[-1:].to_list()
                col2 = df2.columns[:-1].to_list()
                new_col = col1 + col2
                df3 = df2[new_col]

                # Extract data for the current lead time and transpose it
                temp = pd.DataFrame(df3.loc[leadtime]).T

                # Stack data for the first iteration of the outer loop (initialize temp1)
                if year == int(start_year) and month == int(start_month):
                    temp1 = temp
                else:
                    pass

                # Concatenate current iteration's data with existing temp1 DataFrame
                temp1 = pd.concat([temp1, temp], axis=0, ignore_index=True)

        # Remove the first row (header) and rename columns 'mean2' and 'obs2' to 'mean' and 'obs'
        temp1 = temp1.iloc[1:]
        temp1['mean2'] = temp1['mean']
        temp1['obs2'] = temp1['obs']
        temp1 = temp1.drop(['mean', 'obs'], axis=1)
        temp1.rename(columns={'mean2': 'mean', 'obs2': 'obs'}, inplace=True)

        # Set 'date' as the index and save the aggregated data to a CSV file under the 'skill' directory
        temp1.set_index('date', inplace=True)
        temp1.to_csv(path + '/analysis/4.SFFs/3_run/' + folder[bc_type] + '/skill/' + catchment_name + '_' 
                     + str(leadtime) + '_' + folder[bc_type] + '_sffs.csv')

## 4. Calculate CRPS at each lead time

CRPS is a measure of how good forecasts are in matching observed outcomes considering each ensemble. It is a quadratic measure of the difference between the forecast cumulative distribution function (CDF) and the reference dataset of the observation (Zamo and Naveau, 2017). The CRPS is thus calculated as

$$ CRPS= \int [F(x) - H(x > y)]^2 dx $$

where F(x) represents the cumulative distribution of seasonal forecasts, y is observed precipitation, H is called the indicator function which is equals to 1 when x > y and 0 when x < y. Once the CRPS is equals to 0, the forecast is wholly accurate, conversely, the higher the CRPS, the worse the performance of the forecast. 

Also, sometimes we can face the issue from the number of ensemble members. Most of originating centres have changed number of ensemble members once. (Please see A.Download seasonal forecasts datasets / 3. Seasonal forecasts systems and datasets for 8 originating centres / Total precipitation table). In this case, we need to designate exact location and number of ensemble manually. 

This example show you the case when we apply ECMWF datasets to calculate CRPS from 2011 to 2020. In this case, there are 25 ensemble members and 72 rows from Jan.2011 to Dec.2016, also rest of the data have 51 ensemble members. If the number of ensemble members is same, you can put the same number on it.  <font color='red'> Please note that, you need to manually revise some of the code below;</font>

### 4.1 CRPS of ESP

In [71]:
for leadtime in range(1, 8):
    # read ESP data rearranged by lead times
    df = pd.read_csv(path + '/analysis/3.ESP/3_run/skill/' + catchment_name + '_' + str(leadtime) + '_esp.csv')
    
    # Convert DataFrame to numpy array and convert data to float
    df_a = df.to_numpy().astype(float)
    
    # Select ensemble data only from the numpy array
    df_a2 = df_a[:, 1:df_a.shape[1] - 2]
    
    # Select the column for observed data from the numpy array
    df_obs = df_a[:, len(df.columns) - 1]
    
    # Calculate CRPS (Continuous Ranked Probability Score) using observed and ensemble data
    crps_dictionary_rand1 = em.ens_crps(df_obs, df_a2)
    crps = crps_dictionary_rand1['crps']
    
    # Create a DataFrame from CRPS values
    csv = pd.DataFrame(crps)
    
    # Extract month from 'date' column and add it as a new column 'month'
    csv['month'] = df['date'].str.slice(start=5, stop=7)
    
    # Set 'date' as the index of the DataFrame csv
    csv.set_index(df['date'], inplace=True)
    
    # Rename the CRPS column to 'CRPS'
    csv = csv[['month', 0]]
    csv = csv.rename(columns={0: 'CRPS'})
    
    # Add 'leadtime' column to csv DataFrame
    csv['leadtime'] = leadtime
    
    # Stack data: initialize temp DataFrame with csv in the first iteration
    if leadtime == 1:
        temp = csv
    else:
        temp = pd.concat([temp, csv])

# Select columns 'month', 'leadtime', and 'CRPS' from temp DataFrame
temp = temp[['month', 'leadtime', 'CRPS']]

# Save the results to a CSV file under the 'skill' directory for ESP
temp.to_csv(path + '/analysis/3.ESP/3_run/skill/[skill]' + catchment_name + '_esp.csv')

print('The skill (CRPS) of ESP is computed')

The skill (CRPS) of ESP is computed


### 4.2 CRPS of SFFs

In [73]:
import hydrostats.ens_metrics as em   # Import library for calculating CRPS
folder = {1: 'original', 2: 'biascorrected'}

# Should be manually revised (This is an example of ECMWF)) 
num_row = 72     # The number of rows for the first datasets having 'num_col1' of ensemble members
num_col1 = 25    # The number of ensemble members for the first datasets
num_col2 = 51    # The number of ensemble members for the second datasets

for bc_type in range(1, 3):
    for leadtime in range(1, 8):
        # Read SFFs data rearranged by lead times
        df = pd.read_csv(path + '/analysis/4.SFFs/3_run/' + folder[bc_type] + '/skill/' + catchment_name + '_' 
                         + str(leadtime) + '_' + folder[bc_type] + '_sffs.csv')
        
        # Convert DataFrame to numpy array and convert data to float
        df_a = df.to_numpy().astype(float)
        
        # Select ensemble data only
        df_a2 = df_a[:, 1:df_a.shape[1] - 2]
        
        # Select ensemble data having 25 ensembles (~ Dec. 2016)
        df_a3 = df_a2[:num_row, :num_col1]
        
        # Select ensemble data having 51 ensembles (Jan. 2017 ~)
        df_a4 = df_a2[num_row:, :num_col2]
        
        # Select the column for observed data having 25 ensembles (~ Dec. 2016)
        df_obs1 = df_a[:num_row, len(df.columns) - 1]
        
        # Select the column for observed data having 51 ensembles (Jan. 2017 ~)
        df_obs2 = df_a[num_row:, len(df.columns) - 1]
        
        # Calculate CRPS using observed and ensemble data
        crps_dictionary_rand1 = em.ens_crps(df_obs1, df_a3)
        crps_dictionary_rand2 = em.ens_crps(df_obs2, df_a4)
        
        # Extract CRPS values
        temp1 = crps_dictionary_rand1['crps']
        temp2 = crps_dictionary_rand2['crps']
        
        # Concatenate CRPS values along axis 0 (vertically)
        crps = np.concatenate([temp1, temp2], axis=0)
        
        # Create a DataFrame from CRPS values
        csv = pd.DataFrame(crps)
        
        # Extract month from 'date' column and add it as a new column 'month'
        csv['month'] = df['date'].str.slice(start=5, stop=7)
        
        # Add 'date' column to csv DataFrame
        csv['date'] = df['date']
        
        # Set 'date' as the index of the DataFrame csv
        csv.set_index(csv['date'], inplace=True)
        
        # Select columns 'month' and CRPS from csv DataFrame
        csv = csv[['month', 0]]
        csv = csv.rename(columns={0: 'CRPS'})
        
        # Add 'leadtime' column to csv DataFrame
        csv['leadtime'] = leadtime
        
        # Stack data: initialize temp DataFrame with csv in the first iteration
        if leadtime == 1:
            temp = csv
        else:
            temp = pd.concat([temp, csv])
    
    # Select columns 'month', 'leadtime', and 'CRPS' from temp DataFrame
    temp = temp[['month', 'leadtime', 'CRPS']]
    
    # Save the results to a CSV file under the 'skill' directory for SFFs
    temp.to_csv(path + '/analysis/4.SFFs/3_run/' + folder[bc_type] + '/skill/[skill]' + catchment_name 
                + '_' + folder[bc_type] + '_sffs.csv')
    
# Print a message indicating the completion of CRPS computation for SFFs
print('The skill (CRPS) of SFFs has computed.')

The skill (CRPS) of SFFs has computed.


## 5. CRPSS calculation

CRPSS compares the skill of seasonal forecasts with climatology, thus finally it can be simply calculated as 

$$ CRPSS=\ 1\ -\ \frac{{\rm CRPS}^{Sys}}{{\rm CRPS}^{Ref}}$$

where $CRPS^{Sys}$ is previously calculated $CRPS$ (seasonal forecasts), $CRPS^{Ref}$ represents the reference $CRPS$ obtained from climatology. When the skill score is higher (lower) than zero, the forecasting system is more (less) skilful than reference. When it is equal to zero, the system (seasonal forecasts) and the reference (Climatology) have equivalent skill. 

CRPSS can be calculated by runing the code below;

In [74]:
folder = {1: 'original', 2: 'biascorrected'}

for bc_type in range(1, 3):
    # read calculated CRPS data (SFFs)
    df = pd.read_csv(path + f"/analysis/4.SFFs/3_run/{folder[bc_type]}/skill/[skill]{catchment_name}_{folder[bc_type]}_sffs.csv")
    
    # read reference CRPS data (ESP)
    df_ref = pd.read_csv(path + f"/analysis/3.ESP/3_run/skill/[skill]{catchment_name}_esp.csv")
    
    # add reference CRPS column
    df['CRPS_ref'] = df_ref['CRPS']
    
    # add CRPSS column
    df['CRPSS'] = 1 - df['CRPS'] / df['CRPS_ref']
    
    # Initialize 'count' column with zeros
    df['count'] = 0
    
    # Use .loc for assignment based on condition
    df.loc[df['CRPSS'] > 0, 'count'] = 1
    df.loc[df['CRPSS'] <= 0, 'count'] = 0
    
    # Save the updated DataFrame back to the CSV file
    df.to_csv(path + f"/analysis/4.SFFs/3_run/{folder[bc_type]}/skill/[skill]{catchment_name}_{folder[bc_type]}_sffs.csv", index=False)

print('CRPSS calculation has completed.')

CRPSS calculation has completed.
